In [1]:
#Get All the modules needed
from __future__ import print_function,division,unicode_literals,absolute_import

try:
  %tensorflow_version 2.x

except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.layers import Flatten,Dense,Input,AveragePooling2D,Dropout
from keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import os 
import argparse
import numpy as np



Using TensorFlow backend.


In [0]:
#Extract the contents from the th zip folder for data
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/all/Mask_Nonmask/face-mask-detector.zip') as zipex:
  zipex.extractall('/content/drive/My Drive/all/Mask_Nonmask')

In [0]:
mask=os.listdir('/content/drive/My Drive/all/Mask_Nonmask/face-mask-detector/dataset/with_mask')
n_mask=os.listdir('/content/drive/My Drive/all/Mask_Nonmask/face-mask-detector/dataset/without_mask')

In [3]:
data, label=[],[]
x=0
for i in mask:
  imgpath='/content/drive/My Drive/all/Mask_Nonmask/face-mask-detector/dataset/with_mask/'+i
  image=load_img(imgpath,target_size=(299,299))
  image=img_to_array(image)
  image=preprocess_input(image)

  data.append(image)
  label.append('mask')
print("rr")
for i in n_mask:
  imgpath='/content/drive/My Drive/all/Mask_Nonmask/face-mask-detector/dataset/without_mask/'+i
  image=load_img(imgpath,target_size=(299,299))
  image=img_to_array(image)
  image=preprocess_input(image)
 
  data.append(image)
  label.append('non_mask')

data=np.array(data)
label=np.array(label)

print(len(data),len(label))          #print amount of data

rr
1376 1376


In [0]:
lb=LabelBinarizer()
labels=lb.fit_transform(label)
labels=to_categorical(labels)             #similar to one hot encoding

In [0]:
train_x,test_x,train_y,test_y=train_test_split(data,labels,test_size=0.2, stratify=labels)
train_x,val_x,train_y,val_y=train_test_split(train_x,train_y,test_size=0.1 ,stratify=train_y)     #train_validation_test data split

In [0]:
aug=ImageDataGenerator(
    shear_range=0.15,
    rotation_range=20,
    zoom_range=0.15,
    fill_mode="nearest",
    width_shift_range=0.2,
    height_shift_range=0.2

)                                   

In [7]:
test_x[1].shape         #confirm shape of the data array

(299, 299, 3)

In [0]:
base_model=InceptionResNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(299,299,3)))

topmodel=base_model.output
topmodel=AveragePooling2D(pool_size=(4,4))(topmodel)
topmodel=Flatten()(topmodel)
topmodel=Dense(128,activation="relu")(topmodel)
topmodel=Dropout(0.3)(topmodel)
topmodel=Dense(2,activation="relu")(topmodel)


model=Model(inputs=base_model.input,outputs=topmodel)


for layer in base_model.layers:
  layer.trainable=False

                                               # build model

In [0]:
ipt=1e-4

In [0]:
model.compile(loss=keras.losses.BinaryCrossentropy(), 
              optimizer=Adam(learning_rate=ipt),
              metrics=['accuracy'])                             #compile moldel

In [18]:
H=model.fit(aug.flow(train_x,train_y,batch_size=32),
                      steps_per_epoch=len(train_x)//32,
                      validation_data=aug.flow(val_x,val_y,batch_size=32),
                      
                      validation_steps=len(val_x)//32,
                      epochs=5)                                 #Train the model

Epoch 1/5
30/30 [==============================] - 22s 734ms/step - loss: 0.1258 - accuracy: 0.9906 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 2/5
30/30 [==============================] - 19s 649ms/step - loss: 0.1833 - accuracy: 0.9864 - val_loss: 0.1174 - val_accuracy: 0.9688
Epoch 3/5
30/30 [==============================] - 19s 648ms/step - loss: 0.7755 - accuracy: 0.9165 - val_loss: 0.2665 - val_accuracy: 0.9792
Epoch 4/5
30/30 [==============================] - 19s 648ms/step - loss: 0.1671 - accuracy: 0.9843 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 5/5
30/30 [==============================] - 19s 646ms/step - loss: 0.1127 - accuracy: 0.9885 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [20]:
predIdxs = model.predict(test_x, batch_size=32)

predIdxs = np.argmax(predIdxs, axis=1)


print(classification_report(test_y.argmax(axis=1), predIdxs,
	target_names=lb.classes_))                                #Get classification report on test data 

              precision    recall  f1-score   support

        mask       1.00      1.00      1.00       138
    non_mask       1.00      1.00      1.00       138

    accuracy                           1.00       276
   macro avg       1.00      1.00      1.00       276
weighted avg       1.00      1.00      1.00       276



In [23]:
model.save('/content/drive/My Drive/all/Mask_Nonmask/model')              #save model

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/all/Mask_Nonmask/model/assets


In [0]:
from tensorflow.keras.models import load_model                         # load the saved model 
model2=load_model('/content/drive/My Drive/all/Mask_Nonmask/model')

In [ ]:
rest of the snippets are for checking out the model 

In [25]:
predIdxs = model2.predict(test_x, batch_size=32)             # loaded model performance

predIdxs = np.argmax(predIdxs, axis=1)


print(classification_report(test_y.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

              precision    recall  f1-score   support

        mask       1.00      1.00      1.00       138
    non_mask       1.00      1.00      1.00       138

    accuracy                           1.00       276
   macro avg       1.00      1.00      1.00       276
weighted avg       1.00      1.00      1.00       276



In [27]:
model2.summary()              #architechture of the model (very important)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 149, 149, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 149, 149, 32) 0           batch_normalization_203[0][0]    
____________________________________________________________________________________________